# Rex-Omni Fine-tuning on Modal

This notebook demonstrates how to fine-tune the Rex-Omni model using Modal. We will:
1.  Prepare the data.
2.  Upload the data to a Modal Volume.
3.  Trigger the remote training job.
4.  Download the fine-tuned adapter.

In [ ]:
!pip install modal

In [ ]:
import modal
import os

# Authenticate if needed
# !modal token new

## 1. Upload Data

First, we need to upload your `train_data.jsonl` (created using the conversion tool) to the Modal Volume.

In [ ]:
local_data_path = "../lora_finetuning/sample_data.json" # Replace with your actual .jsonl file
remote_data_filename = "train_data.jsonl"
volume_name = "rex-omni-data"

# Create/Get the volume
vol = modal.Volume.from_name(volume_name, create_if_missing=True)

# Upload
with open(local_data_path, "rb") as f:
    vol.write_file(remote_data_filename, f)
    
print(f"Uploaded {local_data_path} to volume '{volume_name}' as '{remote_data_filename}'")

## 2. Run Training

Now we import the training function from our `modal_train.py` script and run it remotely.

In [ ]:
# Import the app and function
# Note: Ensure modal_train.py is in the same directory or python path
from modal_train import train_rex_omni

# Trigger the function
# This will stream logs from the remote GPU
output_path = train_rex_omni.remote(
    data_filename=remote_data_filename,
    num_epochs=3,
    batch_size=1,
    lora_rank=64,
    use_qlora=True
)

print(f"Training finished! Model saved at: {output_path}")

## 3. Download Model

The model is saved in the `rex-omni-models` volume. Let's download the LoRA adapters to our local machine.

In [ ]:
model_volume = modal.Volume.from_name("rex-omni-models")
local_download_dir = "./downloaded_model"
os.makedirs(local_download_dir, exist_ok=True)

# List files in the output directory
# Note: Modal Volume list API might vary, here we just assume we know the path or download the whole dir logic
# For simplicity, we can use the CLI or a helper function to download directory

print(f"Downloading from {output_path} to {local_download_dir}...")

# Currently Modal Python SDK doesn't have a direct 'download directory' method for Volumes easily exposed in one line
# We can use a simple remote function to tar it up and return bytes, or use the CLI.
# Here is a CLI command example you can run in terminal:
# !modal volume get rex-omni-models /path/to/remote/dir ./local_dir

print(f"Run this command in terminal: modal volume get rex-omni-models {output_path} {local_download_dir}")